In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lit,sum,max,min,avg,count,rank,dense_rank,row_number,ntile,lag,lead,first,last
from pyspark.sql import Window


# Create a Spark session
spark = SparkSession.builder.appName("AggregateWindowFunctions").getOrCreate()

# Sample data
data = [
    (1001, '2017-04-01', 'David Smith', 'GuildFord', 10000.00),
    (1002, '2017-04-02', 'David Jones', 'Arlington', 20000.00),
    (1003, '2017-04-03', 'John Smith', 'Shalford', 5000.00),
    (1004, '2017-04-04', 'Michael Smith', 'GuildFord', 15000.00),
    (1005, '2017-04-05', 'David Williams', 'Shalford', 7000.00),
    (1006, '2017-04-06', 'Paum Smith', 'GuildFord', 25000.00),
    (1007, '2017-04-10', 'Andrew Smith', 'Arlington', 15000.00),
    (1008, '2017-04-11', 'David Brown', 'Arlington', 2000.00),
    (1009, '2017-04-20', 'Robert Smith', 'Shalford', 1000.00),
    (1010, '2017-04-25', 'Peter Smith', 'GuildFord', 500.00)
]

# Define the schema
schema = ["order_id", "order_date", "customer_name", "city", "order_amount"]

# Create a DataFrame
df = spark.createDataFrame(data, schema=schema)
df.show(5)

# Aggregation functions
df.agg(
    sum("order_amount").alias("total_order_amount"),
    max("order_amount").alias("max_order_amount"),
    min("order_amount").alias("min_order_amount"),
    avg("order_amount").alias("avg_order_amount"),
    count("*").alias("order_count")
).show(5)

# Window functions
window_spec = Window.orderBy("order_amount")

df.withColumn("rank", rank().over(window_spec)) \
  .withColumn("dense_rank", dense_rank().over(window_spec)) \
  .withColumn("row_number", row_number().over(window_spec)) \
  .withColumn("ntile", ntile(3).over(window_spec)) \
  .withColumn("lag_order_amount", lag("order_amount").over(window_spec)) \
  .withColumn("lead_order_amount", lead("order_amount").over(window_spec)) \
  .withColumn("first_value_order_amount", first("order_amount").over(window_spec)) \
  .withColumn("last_value_order_amount", last("order_amount").over(window_spec)).fillna(0) \
  .show(5)


+--------+----------+--------------+---------+------------+
|order_id|order_date| customer_name|     city|order_amount|
+--------+----------+--------------+---------+------------+
|    1001|2017-04-01|   David Smith|GuildFord|     10000.0|
|    1002|2017-04-02|   David Jones|Arlington|     20000.0|
|    1003|2017-04-03|    John Smith| Shalford|      5000.0|
|    1004|2017-04-04| Michael Smith|GuildFord|     15000.0|
|    1005|2017-04-05|David Williams| Shalford|      7000.0|
+--------+----------+--------------+---------+------------+
only showing top 5 rows

+------------------+----------------+----------------+----------------+-----------+
|total_order_amount|max_order_amount|min_order_amount|avg_order_amount|order_count|
+------------------+----------------+----------------+----------------+-----------+
|          100500.0|         25000.0|           500.0|         10050.0|         10|
+------------------+----------------+----------------+----------------+-----------+

+--------+----

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import col, sum, max, min, avg, count, rank, dense_rank, row_number, ntile, lag, lead, first, last

# Create a Spark session
spark = SparkSession.builder.appName("OrderAnalysis").getOrCreate()

# Assuming you have a DataFrame named orders_df with the provided data
data = [
    (1001, '2017-04-01', 'David Smith', 'GuildFord', 10000.00),
    (1002, '2017-04-02', 'David Jones', 'Arlington', 20000.00),
    (1003, '2017-04-03', 'John Smith', 'Shalford', 5000.00),
    (1004, '2017-04-04', 'Michael Smith', 'GuildFord', 15000.00),
    (1005, '2017-04-05', 'David Williams', 'Shalford', 7000.00),
    (1006, '2017-04-06', 'Paum Smith', 'GuildFord', 25000.00),
    (1007, '2017-04-10', 'Andrew Smith', 'Arlington', 15000.00),
    (1008, '2017-04-11', 'David Brown', 'Arlington', 2000.00),
    (1009, '2017-04-20', 'Robert Smith', 'Shalford', 1000.00),
    (1010, '2017-04-25', 'Peter Smith', 'GuildFord', 500.00)
]

schema = ["order_id", "order_date", "customer_name", "city", "order_amount"]
orders_df = spark.createDataFrame(data, schema=schema)

# Define a window specification based on the 'city' column
window_spec = Window.partitionBy("city").orderBy("order_amount")

# Aggregate functions
agg_df = orders_df.groupBy("city").agg(
    sum("order_amount").alias("total_amount"),
    max("order_amount").alias("max_amount"),
    min("order_amount").alias("min_amount"),
    avg("order_amount").alias("avg_amount"),
    count("order_amount").alias("order_count")
)

# Window functions
window_df = orders_df.withColumn("rank", rank().over(window_spec)) \
    .withColumn("dense_rank", dense_rank().over(window_spec)) \
    .withColumn("row_number", row_number().over(window_spec)) \
    .withColumn("ntile", ntile(3).over(window_spec)) \
    .withColumn("lag_amount", lag("order_amount").over(window_spec)) \
    .withColumn("lead_amount", lead("order_amount").over(window_spec)) \
    .withColumn("first_amount", first("order_amount").over(window_spec)) \
    .withColumn("last_amount", last("order_amount").over(window_spec))

# Show the results
agg_df.show()
window_df.show()


+---------+------------+----------+----------+------------------+-----------+
|     city|total_amount|max_amount|min_amount|        avg_amount|order_count|
+---------+------------+----------+----------+------------------+-----------+
|GuildFord|     50500.0|   25000.0|     500.0|           12625.0|          4|
|Arlington|     37000.0|   20000.0|    2000.0|12333.333333333334|          3|
| Shalford|     13000.0|    7000.0|    1000.0| 4333.333333333333|          3|
+---------+------------+----------+----------+------------------+-----------+

+--------+----------+--------------+---------+------------+----+----------+----------+-----+----------+-----------+------------+-----------+
|order_id|order_date| customer_name|     city|order_amount|rank|dense_rank|row_number|ntile|lag_amount|lead_amount|first_amount|last_amount|
+--------+----------+--------------+---------+------------+----+----------+----------+-----+----------+-----------+------------+-----------+
|    1008|2017-04-11|   David 